## Word Embeddings Using Various Training Strategies

Training a skip-gram model on a small amount of data using the following approaches:
- standard, as explained in the paper: <i>Efficient Estimation of Word Representations in Vector Space</i>
- negative sampling, as explained in the paper: <i>Distributed Representations of Words and Phrases
and their Compositionality</i> and highlighted [here](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling)
- importance sampling, as explained in the paper: <i>On Using Very Large Target Vocabulary for Neural Machine Translation</i>